In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import win32com.client as win32
import sqlite3 as sql
from scipy.optimize import minimize
from scipy.optimize import fsolve
import time

In [2]:
aspen=win32.Dispatch('Apwn.Document')
direccion="C:\\Users\\ASUS\\Documents\\Universidad\\Tesis\\Convergencia_Newton\\Simulación\\equation_oriented.apw"

# def doble_line(dir):
#     return s.replace("\\", "\\\\")

# direccion=doble_line(direccion)
aspen.InitFromArchive2(direccion)
aspen.visible=False
print(f"Aspen cargado")
def correr_simulacion(aspen):
    aspen.Engine.Run2()
    while aspen.Engine.isRunning:
        time.sleep(0.5)



Aspen cargado


In [3]:
def error(x1,x2):
    return abs(x1-x2)
def calculo_error(x,args):
    aspen=args
    flujo,x1,x2=x
    x3=1-x1-x2
    if x1+x2>=1:
        x1=x1/(x1+x2)
        x2=x2/(x1+x2)
        x3=0
    bloques=aspen.Tree.Elements.Item('Data').Elements.Item('Blocks')
    corrientes=aspen.Tree.Elements.Item('Data').Elements.Item('Streams')
    corriente_ingreso=corrientes.Elements.Item('GLR+').Elements.Item("Input")
    Flujo_ingreso=corriente_ingreso.Elements.Item("TOTFLOW").Elements.Item("MIXED")
    Flujo_ingreso.Value=flujo
    concentracion_ingreso=corriente_ingreso.Elements.Item("FLOW").Elements.Item("MIXED")
        
    concentracion_ingreso.Elements.Item("GLYCE-01").Value=x1
    concentracion_ingreso.Elements.Item("WATER").Value=x2
    concentracion_ingreso.Elements.Item("ETHAN-01").Value=x3
    
    correr_simulacion(aspen)
    
    corriente_salida=corrientes.Elements.Item('GLR').Elements.Item("Output")
    flujo_salida=corriente_salida.Elements.Item("TOT_FLOW").Value
    concentracion_salida=corriente_salida.Elements.Item("MOLEFRAC").Elements.Item("MIXED")
    salida_glicerina=concentracion_salida.Elements.Item("GLYCE-01").Value
    salida_agua=concentracion_salida.Elements.Item("WATER").Value
    salida_etanol=concentracion_salida.Elements.Item("ETHAN-01").Value
    error_glicerina=error(salida_glicerina,x1)
    error_agua=error(salida_agua,1-x1)
    error_flujo=error(flujo_salida,flujo)
    error_etanol=error(salida_etanol,1-x1-x2)
    # return error_glicerina,error_agua,error_flujo
    norma=np.linalg.norm([error_glicerina,error_agua,error_flujo,error_etanol])
    return norma
    # norma_concentracion=np.linalg.norm([error_glicerina,error_agua,error_etanol])
    # return error_flujo,error_glicerina,error_agua
    
    

In [4]:
# correr_simulacion(aspen)
vector=[99.8,0.9999,0.0001]
args=aspen
limites=[(0,99.9999),(0.98,1),(0,0.02)]
print(calculo_error(vector,args))
maxiter=100
result=minimize(calculo_error,vector,args=args,bounds=limites,options={'maxiter':maxiter})
# result=fsolve(calculo_error,vector,args=args,xtol=1e-6)
print(result)

1.146889008637256
  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 0.02432500245518916
        x: [ 9.865e+01  9.999e-01  1.026e-04]
      nit: 5
      jac: [ 1.000e+00  6.575e-07  8.397e-03]
     nfev: 140
     njev: 35
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>


In [5]:
print(calculo_error(result.x,args))

0.02432500245521985


In [6]:
aspen.Close()